In [77]:
# %%file strategy_1.py
from collections import OrderedDict

import import_ipynb
from strategy_base import TradeStrategyBase

import pandas as pd
import numpy as np

class TradeStrategy1(TradeStrategyBase):
    """
    Trade Strategy 1: Following the trend.
    When price rises or falls above an absolute threshold x percent (p_change_threshold), default to 2%. 
    Buy and hold for a number of days (s_keep_stock_threshold), default to  20. 
    """
    _p_change_threshold = 0.02
    _keep_stock_threshold = 20
    _trade_profit_array = []
    
    def __init__(self, p_change_threshold:float = 0.02, s_keep_stock_threshold:int = 20):
        """
        Initialize Strategy with given parameters
        :type p_change_threshold: float
        :type s_keep_stock_threshold: int 
        :param p_change_threshold: Absolute threshold for a stock to rise or fall through in a day. E.g. 2% = 0.02 
        :param s_keep_stock_threshold: Number of days to hold a stock. E.g. 20 days = 20
        """
        self._p_change_threshold = p_change_threshold
        self._keep_stock_threshold = s_keep_stock_threshold
        
        self._held_period_days = 0
        
    def trade(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Perform Analysis, Decide on Trade, and Execute Trade
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        :return: 
        """
        
        """
        First, Buying
        """
        if self._evaluate_buying(date=date,
                                 tradeday=tradeday):
            self._execute_buying(date=date,
                                 tradeday=tradeday)
        
        """
        Secondly, Selling
        """
        if self._evaluate_selling(date=date,
                                  tradeday=tradeday):
            self._execute_selling(date=date,
                                  tradeday=tradeday)
            
        """
        Lastly, Increment holding date count
        """
        if self._held_period_days != 0:
            # Is currently holding, attach p_change as profit
            self._trade_profit_array.append(tradeday.p_change)
            # Increment date
            self._held_period_days += 1
        
        
    def _evaluate_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        trade_day: pd.Series = tradeday.copy() # Create a copy() in case we need to change it.
        if self._held_period_days == 0 and \
                np.abs(trade_day.p_change) > self._p_change_threshold:
            # When there is no holding, and rise or fall in price had reached the above pre_defined _p_change_threshold
            return True
        else:
            return False
        
    def _execute_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        print("Bought on {} at ${}".format(date, tradeday.close))
        self._held_period_days += 1 # Start holding date count

    def _evaluate_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._held_period_days >= self._keep_stock_threshold:
            # When holding period exceeds s_keep_stock_threshold, sell stocks
            return True
        else:
            return False
            
    def _execute_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        print("Sold on {} at ${}. So far made {}%".format(date, tradeday.close, round(self.trade_profit, 4)))
        self._held_period_days = 0  # Reset holding count 
        
    @property
    def trade_profit(self):
        return np.sum(self._trade_profit_array)
    
    @property
    def p_change_threshold(self):
        return self._p_change_threshold
    
    @property
    def keep_stock_threshold(self):
        return self._keep_stock_threshold

In [78]:
amex_df = pd.read_csv('../../data/amex_df.csv',
                           parse_dates=True,
                           index_col=0)
strategy = TradeStrategy1()
for date, tradeday in amex_df.iterrows():
    strategy.trade(date=date,
                   tradeday=tradeday)
print("Strategy Yielded: {}%".format(strategy.trade_profit))

Bought on 2017-06-05 00:00:00 at $78.97
Sold on 2017-06-30 00:00:00 at $84.24. So far made 6.375%
Bought on 2017-07-03 00:00:00 at $85.1
Sold on 2017-07-31 00:00:00 at $85.23. So far made 7.088%
Bought on 2017-08-02 00:00:00 at $85.07
Sold on 2017-08-29 00:00:00 at $85.42. So far made 7.338%
Bought on 2017-08-30 00:00:00 at $85.69
Sold on 2017-09-27 00:00:00 at $89.5. So far made 11.192%
Bought on 2017-09-28 00:00:00 at $90.14
Sold on 2017-10-25 00:00:00 at $93.53. So far made 15.99%
Bought on 2017-10-26 00:00:00 at $95.69
Sold on 2017-11-22 00:00:00 at $93.82. So far made 16.98%
Bought on 2017-11-24 00:00:00 at $93.48
Sold on 2017-12-21 00:00:00 at $98.5. So far made 21.936%
Bought on 2017-12-22 00:00:00 at $98.74
Sold on 2018-01-23 00:00:00 at $97.98. So far made 21.497%
Bought on 2018-01-24 00:00:00 at $99.3
Sold on 2018-02-21 00:00:00 at $97.5. So far made 21.233%
Bought on 2018-02-22 00:00:00 at $97.35
Sold on 2018-03-21 00:00:00 at $95.31. So far made 18.836%
Bought on 2018-03-22